<a href="https://colab.research.google.com/github/ReynaraEzra/NLP-Tensorflow/blob/main/Multiclass_Classification_NLP_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print(tf.__version__)

2.5.0


In [2]:
!pip install ipython-autotime
%load_ext autotime

time: 160 µs (started: 2021-06-30 14:40:48 +00:00)


In [3]:
import pandas as pd
path = '/content/imdb_indonesian_movies_2.csv'
df = pd.read_csv(path)

time: 29.7 ms (started: 2021-06-30 14:40:48 +00:00)


In [4]:
df.head()

,judul_film,ringkasan_sinopsis,genre
0,Sunan Kalijaga,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Gie,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL Movie,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Sang pencerah,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


time: 26.3 ms (started: 2021-06-30 14:40:48 +00:00)


In [5]:
category = pd.get_dummies(df['genre'])
category.head()

,Drama,Horor,Komedi,Laga,Romantis
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0


time: 21.7 ms (started: 2021-06-30 14:40:48 +00:00)


In [6]:
df = df.drop(['judul_film', 'genre'], axis=1)

time: 7.15 ms (started: 2021-06-30 14:40:48 +00:00)


In [7]:
df.head()

,ringkasan_sinopsis
0,Raden Mas Said putra sulung Tumenggung Wilarik...
1,Soe Hok Gie adalah seorang aktivis yang hidup ...
2,Guru Bangsa Tjokroaminoto menceritakan tentang...
3,POL menceritakan kisah hidup yang luar biasa d...
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...


time: 16.1 ms (started: 2021-06-30 14:40:48 +00:00)


In [8]:
df_new = pd.concat([df, category], axis=1)

time: 3.54 ms (started: 2021-06-30 14:40:48 +00:00)


In [9]:
df_new.head()

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0


time: 30.4 ms (started: 2021-06-30 14:40:48 +00:00)


In [10]:
ringkasan = df_new['ringkasan_sinopsis'].values
label = df_new[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values

time: 4.16 ms (started: 2021-06-30 14:40:48 +00:00)


In [11]:
from sklearn.model_selection import train_test_split

ringkasan_train, ringkasan_test, label_train, label_test = train_test_split(
    ringkasan, 
    label,
    test_size=0.2,
    random_state=0
)

time: 824 ms (started: 2021-06-30 14:40:48 +00:00)


In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token='-')
tokenizer.fit_on_texts(ringkasan_train)
tokenizer.fit_on_texts(ringkasan_test)

sequence_train = tokenizer.texts_to_sequences(ringkasan_train)
sequence_test = tokenizer.texts_to_sequences(ringkasan_test)

pad_train = pad_sequences(sequence_train)
pad_test = pad_sequences(sequence_test)

time: 227 ms (started: 2021-06-30 14:40:49 +00:00)


In [13]:
model = tf.keras.models.Sequential([
   tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
   tf.keras.layers.LSTM(64),
   tf.keras.layers.Dense(128, activation='relu'),
   tf.keras.layers.Dense(64, activation='relu'),
   tf.keras.layers.Dense(5, activation='softmax')
])

time: 693 ms (started: 2021-06-30 14:40:49 +00:00)


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          80000     
_________________________________________________________________
lstm (LSTM)                  (None, 64)                20736     
_________________________________________________________________
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
Total params: 117,637
Trainable params: 117,637
Non-trainable params: 0
_________________________________________________________________
time: 4.82 ms (started: 2021-06-30 14:40:50 +00:00)


In [15]:
model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

time: 21.4 ms (started: 2021-06-30 14:40:50 +00:00)


In [16]:
history = model.fit(
    pad_train,
    label_train,
    epochs=20,
    validation_data=(pad_test, label_test),
    verbose=2
)

Epoch 1/20
26/26 - 26s - loss: 1.6096 - accuracy: 0.2002 - val_loss: 1.6159 - val_accuracy: 0.1294
Epoch 2/20
26/26 - 23s - loss: 1.6029 - accuracy: 0.2189 - val_loss: 1.6209 - val_accuracy: 0.1343
Epoch 3/20
26/26 - 23s - loss: 1.5683 - accuracy: 0.3221 - val_loss: 1.6099 - val_accuracy: 0.1642
Epoch 4/20
26/26 - 24s - loss: 1.3798 - accuracy: 0.4540 - val_loss: 1.6158 - val_accuracy: 0.2388
Epoch 5/20
26/26 - 23s - loss: 0.9875 - accuracy: 0.5896 - val_loss: 2.0377 - val_accuracy: 0.2836
Epoch 6/20
26/26 - 23s - loss: 0.6555 - accuracy: 0.7065 - val_loss: 2.1689 - val_accuracy: 0.2736
Epoch 7/20
26/26 - 23s - loss: 0.4216 - accuracy: 0.8470 - val_loss: 2.5555 - val_accuracy: 0.3134
Epoch 8/20
26/26 - 23s - loss: 0.2113 - accuracy: 0.9440 - val_loss: 3.0206 - val_accuracy: 0.2786
Epoch 9/20
26/26 - 23s - loss: 0.0890 - accuracy: 0.9813 - val_loss: 3.1102 - val_accuracy: 0.2886
Epoch 10/20
26/26 - 23s - loss: 0.0261 - accuracy: 0.9975 - val_loss: 3.8245 - val_accuracy: 0.2935
Epoch 11/